# Ollama를 사용해 RAG 구성

### 왜 Ollama인가?
> Ollama는 CPU만으로도 LLM을 실행할 수 있도록 설계된 경량화된 모델을 제공하기 때문에 GPU가 필수가 아니기 때문 <br>
> 오픈소스 이므로 돈이 안듦 <br>

------------------------------------------------------------------------------------------------
- docx 문서를 준비한다.
- docx 문서를 쪼갠다.
- Ollama Text Embedding Model을 사용해서 임베딩한다.
- Pinecone 벡터 데이터베이스에 임베딩한다.
- 질문에 대한 유사도 검사를 실시해서 적절한 문서 내용을 받아본다.
- Gemma LLM으로 질문에 대한 응답을 받는다.
- prompt는 langchain에서 제공하는 RetrievalQA를 사용한다.
- 질문에 대한 질의를 받는다

In [10]:
%pip install --upgrade --quiet  docx2txt langchain-community langchain-text-splitters langchain_ollama langchain-pinecone langchain langchainhub

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200
)

loader = Docx2txtLoader("./choichiwon.docx")
documents = loader.load_and_split(text_splitter=text_splitter)

In [3]:
len(documents)

1

In [4]:
from langchain_ollama import OllamaEmbeddings

embedding = OllamaEmbeddings(model="gemma2")

In [6]:
from dotenv import load_dotenv
from langchain_pinecone import PineconeVectorStore

load_dotenv()

index_name= 'ollama-choichiwon'

database = PineconeVectorStore.from_documents(documents, embedding, index_name=index_name)

In [7]:
query = '최치원이란 사람에 대해 설명해줘'

retrieved_docs = database.similarity_search(query, k=2)

In [8]:
retrieved_docs

[Document(id='ed8151fd-84b0-441a-8b11-9b8d5f987a67', metadata={'source': './choichiwon.docx'}, page_content='최치원은 1995년 6월 29일에 태어난 30살 대한민국 남성으로 현재 2024년 9월 18일 대한민국의 오픈소스컨설팅이라는 회사에 다니고 있으며 웹 애플리케이션 개발자이고 아버지는 최인석 어머니는 이혜정이다.')]

In [9]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="gemma2")

In [11]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

/Users/choichiwon/.pyenv/versions/helloworld/lib/python3.10/site-packages/langsmith/client.py:5301: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


In [12]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(search_kwargs={'k': 2}),
    chain_type_kwargs={"prompt": prompt}
)

In [13]:
ai_message = qa_chain({"query": query})

/var/folders/54/2vnvnn154r5_5fwkj7dvqtn00000gn/T/ipykernel_89342/3455095564.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  ai_message = qa_chain({"query": query})


In [14]:
ai_message

{'query': '최치원이란 사람에 대해 설명해줘',
 'result': '최치원은 1995년 6월 29일에 태어난 대한민국 남성입니다.  그는 현재 30세이며 웹 애플리케이션 개발자로 일하고 있습니다. 최치원의 아버지는 최인석이고 어머니는 이혜정입니다.  \n'}